In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Behavioral RL Experiment Analysis\n",
    "\n",
    "This notebook analyzes the results of our experiments comparing risk-sensitive RL agents with baseline agents."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "\n",
    "# Set style\n",
    "plt.style.use('seaborn')\n",
    "sns.set_palette('husl')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load metrics\n",
    "baseline_metrics = pd.read_csv('../results/metrics/baseline_metrics.csv')\n",
    "risk_sensitive_metrics = pd.read_csv('../results/metrics/risk_sensitive_metrics.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Performance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_performance_comparison(baseline_df, risk_sensitive_df):\n",
    "    \"\"\"Plot key performance metrics comparison\"\"\"\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "    \n",
    "    # Final capital distribution\n",
    "    sns.boxplot(data=[baseline_df['final_capital'], risk_sensitive_df['final_capital']], \n",
    "                ax=axes[0,0])\n",
    "    axes[0,0].set_xticklabels(['Baseline', 'Risk-Sensitive'])\n",
    "    axes[0,0].set_title('Final Capital Distribution')\n",
    "    \n",
    "    # Risk metrics\n",
    "    metrics = ['sharpe_ratio', 'risk_seeking_ratio']\n",
    "    positions = [(0,1), (1,0)]\n",
    "    \n",
    "    for metric, pos in zip(metrics, positions):\n",
    "        data = [baseline_df[metric], risk_sensitive_df[metric]]\n",
    "        sns.boxplot(data=data, ax=axes[pos])\n",
    "        axes[pos].set_xticklabels(['Baseline', 'Risk-Sensitive'])\n",
    "        axes[pos].set_title(f'{metric.replace(\"_\", \" \").title()}')\n",
    "    \n",
    "    # Statistical tests\n",
    "    axes[1,1].axis('off')\n",
    "    stats_text = []\n",
    "    for metric in ['final_capital', 'sharpe_ratio', 'risk_seeking_ratio']:\n",
    "        t_stat, p_val = stats.ttest_ind(baseline_df[metric], risk_sensitive_df[metric])\n",
    "        stats_text.append(f'{metric}:\\nt={t_stat:.2f}, p={p_val:.3f}')\n",
    "    \n",
    "    axes[1,1].text(0.1, 0.5, '\\n\\n'.join(stats_text), fontsize=12)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    return fig"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Behavioral Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_behavioral_patterns(baseline_df, risk_sensitive_df):\n",
    "    \"\"\"Plot behavioral patterns comparison\"\"\"\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(12, 12))\n",
    "    \n",
    "    # Investment after gains vs losses\n",
    "    conditions = ['after_gains', 'after_losses']\n",
    "    baseline_data = [baseline_df[f'investment_{c}'] for c in conditions]\n",
    "    risk_data = [risk_sensitive_df[f'investment_{c}'] for c in conditions]\n",
    "    \n",
    "    x = np.arange(len(conditions))\n",
    "    width = 0.35\n",
    "    \n",
    "    axes[0].bar(x - width/2, [np.mean(d) for d in baseline_data], width, \n",
    "                yerr=[np.std(d) for d in baseline_data], label='Baseline')\n",
    "    axes[0].bar(x + width/2, [np.mean(d) for d in risk_data], width,\n",
    "                yerr=[np.std(d) for d in risk_data], label='Risk-Sensitive')\n",
    "    \n",
    "    axes[0].set_xticks(x)\n",
    "    axes[0].set_xticklabels(['After Gains', 'After Losses'])\n",
    "    axes[0].set_title('Investment Behavior')\n",
    "    axes[0].legend()\n",
    "    \n",
    "    # Risk preference evolution\n",
    "    time_steps = range(len(baseline_df['risk_preference_evolution'].iloc[0]))\n",
    "    baseline_risk = np.mean([eval(x) for x in baseline_df['risk_preference_evolution']], axis=0)\n",
    "    risk_sensitive_risk = np.mean([eval(x) for x in risk_sensitive_df['risk_preference_evolution']], axis=0)\n",
    "    \n",
    "    axes[1].plot(time_steps, baseline_risk, label='Baseline')\n",
    "    axes[1].plot(time_steps, risk_sensitive_risk, label='Risk-Sensitive')\n",
    "    axes[1].set_title('Risk Preference Evolution')\n",
    "    axes[1].set_xlabel('Time Step')\n",
    "    axes[1].set_ylabel('Risk Preference')\n",
    "    axes[1].legend()\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    return fig"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Generate Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def generate_report(baseline_df, risk_sensitive_df):\n",
    "    \"\"\"Generate a comprehensive analysis report\"\"\"\n",
    "    # Performance metrics\n",
    "    metrics = ['final_capital', 'sharpe_ratio', 'risk_seeking_ratio']\n",
    "    \n",
    "    print(\"Performance Comparison:\")\n",
    "    print(\"-\" * 50)\n",
    "    for metric in metrics:\n",
    "        baseline_mean = baseline_df[metric].mean()\n",
    "        baseline_std = baseline_df[metric].std()\n",
    "        risk_mean = risk_sensitive_df[metric].mean()\n",
    "        risk_std = risk_sensitive_df[metric].std()\n",
    "        \n",
    "        print(f\"{metric.replace('_', ' ').title()}:\")\n",
    "        print(f\"Baseline: {baseline_mean:.3f} ± {baseline_std:.3f}\")\n",
    "        print(f\"Risk-Sensitive: {risk_mean:.3f} ± {risk_std:.3f}\")\n",
    "        \n",
    "        # Statistical test\n",
    "        t_stat, p_val = stats.ttest_ind(baseline_df[metric], risk_sensitive_df[metric])\n",
    "        print(f\"t-statistic: {t_stat:.3f}, p-value: {p_val:.3f}\\n\")\n",
    "    \n",
    "    # Save results\n",
    "    fig1 = plot_performance_comparison(baseline_df, risk_sensitive_df)\n",
    "    fig2 = plot_behavioral_patterns(baseline_df, risk_sensitive_df)\n",
    "    \n",
    "    fig1.savefig('../results/figures/performance_comparison.png')\n",
    "    fig2.savefig('../results/figures/behavioral_patterns.png')\n",
    "    \n",
    "    # Update README\n",
    "    with open('../README.md', 'r') as f:\n",
    "        readme = f.read()\n",
    "    \n",
    "    # Update results section\n",
    "    results_table = f\"\"\"\n",
    "| Metric | Baseline | Risk-Sensitive | % Difference |\n",
    "|--------|----------|----------------|------------|\n",
    "| Mean Final Capital | {baseline_df['final_capital'].mean():.2f} ± {baseline_df['final_capital'].std():.2f} | {risk_sensitive_df['final_capital'].mean():.2f} ± {risk_sensitive_df['final_capital'].std():.2f} | {((risk_sensitive_df['final_capital'].mean() - baseline_df['final_capital'].mean()) / baseline_df['final_capital'].mean() * 100):.1f}% |\n",
    "| Sharpe Ratio | {baseline_df['sharpe_ratio'].mean():.2f} ± {baseline_df['sharpe_ratio'].std():.2f} | {risk_sensitive_df['sharpe_ratio'].mean():.2f} ± {risk_sensitive_df['sharpe_ratio'].std():.2f} | {((risk_sensitive_df['sharpe_ratio'].mean() - baseline_df['sharpe_ratio'].mean()) / baseline_df['sharpe_ratio'].mean() * 100):.1f}% |\n",
    "| Risk-Seeking Ratio | {baseline_df['risk_seeking_ratio'].mean():.2f} ± {baseline_df['risk_seeking_ratio'].std():.2f} | {risk_sensitive_df['risk_seeking_ratio'].mean():.2f} ± {risk_sensitive_df['risk_seeking_ratio'].std():.2f} | {((risk_sensitive_df['risk_seeking_ratio'].mean() - baseline_df['risk_seeking_ratio'].mean()) / baseline_df['risk_seeking_ratio'].mean() * 100):.1f}% |\n",
    "\"\"\"\n",
    "    \n",
    "    # Update README with new results\n",
    "    import re\n",
    "    from datetime import datetime\n",
    "    \n",
    "    date_str = datetime.now().strftime(\"%Y-%m-%d\")\n",
    "    readme = re.sub(r'\\[DATE\\]', date_str, readme)\n",
    "    readme = re.sub(r'\\| Mean Final Capital \\|.*\\|\\n\\| Sharpe Ratio \\|.*\\|\\n\\| Risk-Seeking Ratio \\|.*\\|', results_table.strip(), readme)\n",
    "    \n",
    "    with open('../README.md', 'w') as f:\n",
    "        f.write(readme)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}